# Import data and libraries

In [386]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn import preprocessing as p
from sklearn import tree
import graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.cluster import KMeans 
telco = pd.read_csv('telco.csv')

# Dataset overview

In [408]:
print ("Number of records: ",telco.shape[0])
print ("Number of columns: " ,telco.shape[1])
print ("Features: \n",telco.columns.tolist())
print ("Number of missing values: \n", telco.isnull().sum())

Number of records:  7032
Number of columns:  21
Features: 
 ['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']
Number of missing values: 
 customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64


In [388]:
print("Number of unique value: \n", telco.nunique())

Number of unique value: 
 customerID          7043
gender                 2
SeniorCitizen          2
Partner                2
Dependents             2
tenure                73
PhoneService           2
MultipleLines          3
InternetService        3
OnlineSecurity         3
OnlineBackup           3
DeviceProtection       3
TechSupport            3
StreamingTV            3
StreamingMovies        3
Contract               3
PaperlessBilling       2
PaymentMethod          4
MonthlyCharges      1585
TotalCharges        6531
Churn                  2
dtype: int64


In [389]:
print("Data types for each feature: \n",telco.dtypes)

Data types for each feature: 
 customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object


In [390]:
telco

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


# Data manipulation

In [419]:
# replace spaces with null values in total charges column
telco['TotalCharges'] = telco["TotalCharges"].replace(" ",np.nan)

# drop null values from total charges column
telco = telco[telco["TotalCharges"].notnull()]
telco = telco.reset_index()[telco.columns]

In [420]:
# convert to float type
telco["TotalCharges"] = telco["TotalCharges"].astype(float)
telco["MonthlyCharges"] = telco["MonthlyCharges"].astype(float)
telco['tenure'] = telco['tenure'].astype(float)

In [421]:
# store customerID and Churn column names 
custid = ['customerID']
target = ['Churn']

In [422]:
# store categorical column names 
categorical = telco.nunique()[telco.nunique() < 5].keys().tolist()

#remove target from the list of categorical variables 
categorical.remove(target[0])

In [423]:
# store numerical column names
numerical = [x for x in telco.columns if x not in custid + target + categorical]

In [424]:
# one-hot encode categorical variables 
telco = pd.get_dummies(data = telco, columns = categorical, drop_first = True)

In [425]:
telco

,customerID,tenure,MonthlyCharges,TotalCharges,Churn,gender_Male,SeniorCitizen_1,Partner_Yes,Dependents_Yes,PhoneService_Yes,...,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,7590-VHVEG,1.0,29.85,29.85,No,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0
1,5575-GNVDE,34.0,56.95,1889.50,No,1,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1
2,3668-QPYBK,2.0,53.85,108.15,Yes,1,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1
3,7795-CFOCW,45.0,42.30,1840.75,No,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,9237-HQITU,2.0,70.70,151.65,Yes,0,0,0,0,1,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7027,6840-RESVB,24.0,84.80,1990.50,No,1,0,1,1,1,...,0,1,0,1,1,0,1,0,0,1
7028,2234-XADUH,72.0,103.20,7362.90,No,0,0,1,1,1,...,0,1,0,1,1,0,1,1,0,0
7029,4801-JZAZL,11.0,29.60,346.45,No,0,0,1,1,0,...,0,0,0,0,0,0,1,0,1,0
7030,8361-LTMKD,4.0,74.40,306.60,Yes,1,1,1,0,1,...,0,0,0,0,0,0,1,0,0,1


# Train/test dataset split

In [427]:
# print churn values
print(set(telco['Churn']))

{'No', 'Yes'}


In [428]:
# investigate the distribution of each churn group (should adjust if one group < 5%)
telco.groupby(['Churn']).size() / telco.shape[0] * 100

Churn
No     73.421502
Yes    26.578498
dtype: float64

In [429]:
# load independent and dependent features
X = [i for i in telco.columns if i not in custid + target]
Y = target

In [458]:
Y.ravel()

AttributeError: 'list' object has no attribute 'ravel'

In [430]:
# split data into 25% to testing and 75% to training 
train, test = train_test_split(telco, test_size=0.25)
train_X = train[X]
train_Y = train[Y]
test_X = test[X]
test_Y = test[Y]

# Predict churn rate with logistic regression

In [431]:
# fit logistic regression on training data 
logreg = LogisticRegression()
logreg.fit(train_X, train_Y)

C:\Users\Erica\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Erica\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [432]:
# predict churn labels on testing data 
pred_test_Y = logreg.predict(test_X)

In [433]:
# calculate accuracy score on testing data 
test_accuracy = accuracy_score(test_Y, pred_test_Y)
print('Test accuracy:', round(test_accuracy, 4))

Test accuracy: 0.806


# Logistic regression with L1 regularization (Lasso Regression)

In [464]:
#identify the optimal C for highest test accuracy
C = [1,0.5,0.25,0.1,0.05,0.025,0.01,0.005,0.0025]
for C_value in C:
    logreg = LogisticRegression(penalty='l1', C=C_value, solver='liblinear')
    logreg.fit(train_X, train_Y)
    pred_test_Y = logreg.predict(test_X)
    test_accuracy = accuracy_score(test_Y, pred_test_Y)
    print('C: ', C_value)
    print('Test accuracy:', round(test_accuracy, 4))

C:\Users\Erica\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Erica\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Erica\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


C:  1
Test accuracy: 0.8055
C:  0.5
Test accuracy: 0.8066
C:  0.25
Test accuracy: 0.806


C:\Users\Erica\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Erica\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Erica\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Erica\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

C:  0.1
Test accuracy: 0.8043
C:  0.05
Test accuracy: 0.8055
C:  0.025
Test accuracy: 0.806
C:  0.01
Test accuracy: 0.789
C:  0.005
Test accuracy: 0.7753
C:  0.0025
Test accuracy: 0.7759


C:\Users\Erica\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Erica\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Predict churn rate with decision tree

In [472]:
# identify the optimal max_depth for highest test accuracy
depth_list = list(range(2,15))

for max_depth_value in depth_list:
    mytree = DecisionTreeClassifier(max_depth=max_depth_value)
    mytree.fit(train_X, train_Y)
    pred_test_Y = mytree.predict(test_X)
    test_accuracy = accuracy_score(test_Y, pred_test_Y)
    print('max_depth: ', max_depth_value)
    print('Test accuracy:', round(test_accuracy, 4))

max_depth:  2
Test accuracy: 0.7873
max_depth:  3
Test accuracy: 0.7873
max_depth:  4
Test accuracy: 0.7867
max_depth:  5
Test accuracy: 0.7878
max_depth:  6
Test accuracy: 0.7747
max_depth:  7
Test accuracy: 0.7725
max_depth:  8
Test accuracy: 0.7702
max_depth:  9
Test accuracy: 0.76
max_depth:  10
Test accuracy: 0.7594
max_depth:  11
Test accuracy: 0.7617
max_depth:  12
Test accuracy: 0.7497
max_depth:  13
Test accuracy: 0.7366
max_depth:  14
Test accuracy: 0.7338
